In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import shutil
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files
from tensorflow.keras import mixed_precision

# Enable mixed precision training for performance boost
mixed_precision.set_global_policy("mixed_float16")

# Upload Kaggle API key
files.upload()

# Setup Kaggle API
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download Paul Timothy Mooney Chest X-ray Pneumonia dataset
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia --unzip -p /content/dataset

# Define dataset paths
data_dir = "/content/dataset/chest_xray"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")
val_dir = os.path.join(data_dir, "val")

# Image Preprocessing and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(128, 128), batch_size=16, class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir, target_size=(128, 128), batch_size=16, class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir, target_size=(128, 128), batch_size=16, class_mode='binary'
)

# Spatial Transformer Network (STN) Layer
def build_stn(input_shape=(128, 128, 3)):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(50, activation='relu')(x)
    x = layers.Dense(6, activation='linear')(x)
    return keras.Model(inputs, x)

# Build VGG16 Model with STN
base_model = keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False

# Unfreeze last 4 layers for fine-tuning
for layer in base_model.layers[-4:]:
    layer.trainable = True

inputs = keras.Input(shape=(128, 128, 3))
stn = build_stn()(inputs)
x = base_model(inputs, training=False)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, x)

# Compile Model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train Model
checkpoint = keras.callbacks.ModelCheckpoint("chest_xray_model.h5", save_best_only=True)
history = model.fit(train_generator, epochs=10, validation_data=val_generator, callbacks=[checkpoint])

# Evaluate Model
test_loss, test_acc = model.evaluate(test_generator)
print(f"✅ Test Accuracy: {test_acc:.4f}")

# Predict on test data
y_true = test_generator.classes  # Actual labels
y_pred = (model.predict(test_generator) > 0.5).astype(int)  # Predicted labels (Threshold at 0.5)

# Compute Precision, Recall, and F1-score
print("\n📊 Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Normal', 'Pneumonia']))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Pneumonia'], yticklabels=['Normal', 'Pneumonia'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
100% 2.29G/2.29G [01:44<00:00, 23.9MB/s]
100% 2.29G/2.29G [01:44<00:00, 23.7MB/s]
Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - accuracy: 0.8981 - loss: 0.2659

326/326 ━━━━━━━━━━━━━━━━━━━━ 72s 194ms/step - accuracy: 0.8982 - loss: 0.2657 - val_accuracy: 0.8750 - val_loss: 0.2448
Epoch 2/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 61s 187ms/step - accuracy: 0.9529 - loss: 0.1170 - val_accuracy: 0.7500 - val_loss: 0.4292
Epoch 3/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 62s 189ms/step - accuracy: 0.9683 - loss: 0.0930 - val_accuracy: 0.8125 - val_loss: 0.3069
Epoch 4/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - accuracy: 0.9684 - loss: 0.0848

326/326 ━━━━━━━━━━━━━━━━━━━━ 61s 188ms/step - accuracy: 0.9684 - loss: 0.0848 - val_accuracy: 0.9375 - val_loss: 0.2214
Epoch 5/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - accuracy: 0.9745 - loss: 0.0706

326/326 ━━━━━━━━━━━━━━━━━━━━ 61s 187ms/step - accuracy: 0.9745 - loss: 0.0706 - val_accuracy: 0.8750 - val_loss: 0.1998
Epoch 6/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.9787 - loss: 0.0517

326/326 ━━━━━━━━━━━━━━━━━━━━ 61s 188ms/step - accuracy: 0.9787 - loss: 0.0517 - val_accuracy: 1.0000 - val_loss: 0.1132
Epoch 7/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 61s 186ms/step - accuracy: 0.9791 - loss: 0.0544 - val_accuracy: 0.8125 - val_loss: 0.4592
Epoch 8/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 61s 186ms/step - accuracy: 0.9822 - loss: 0.0487 - val_accuracy: 0.7500 - val_loss: 0.5622
Epoch 9/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 60s 186ms/step - accuracy: 0.9826 - loss: 0.0497 - val_accuracy: 1.0000 - val_loss: 0.1161
Epoch 10/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 61s 187ms/step - accuracy: 0.9825 - loss: 0.0491 - val_accuracy: 0.8750 - val_loss: 0.3179
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.8299 - loss: 0.9004
Test Accuracy: 0.8446
